In [2]:
from py_ecc.bn128 import G1, G2, multiply, add, curve_order, eq, Z1, pairing, neg, FQ, final_exponentiate, FQ12
import numpy as np
import galois   

In [3]:
GF = galois.GF(curve_order)
# takes about 2 minutes

In [4]:
# This script demonstrates the polynomial evaluations with powers of tau
# we see that evaluating a polyomial at tau is the same as computing the inner product of the polynomial coefficients and the powers of tau

tau_testing = GF(20)

# (4x^3 + 2x^2 + 3x + 1) * (x+2)
polynomial = galois.Poly([4,2,3,1], field=GF) * galois.Poly([1,2], field=GF)
print(f"Polynomial: {polynomial}")  # Outputs: 4x^4 + 10x^3 + 7x^2 + 7x + 2

# Evaluate the polynomial at tau.
p_eval_at_tau = polynomial(tau_testing)
print(f"Polynomial evaluated at tau: {p_eval_at_tau}") # Outputs: 3962
p_eval_at_tau_G1 = multiply(G1, int(p_eval_at_tau))

def powers_of_tau_test(tau, degree):
    powers_of_tau_testing = []
    for i in range(degree+1):
        print(f"i = {i}, tau**i = {tau**i}")
        powers_of_tau_testing.append(multiply(G1, int(tau**i)))
    return powers_of_tau_testing

def compute_powers_of_tau_test(tau, degree):
    """Computes and prints the powers of tau up to the specified degree and returns them
    as elliptic curve points by multiplying each power by the base point G1."""
    powers_of_tau = []
    for i in range(degree + 1):
        tau_power = tau ** i
        tau_power_G1 = multiply(G1, int(tau_power))
        powers_of_tau.append(tau_power_G1)
        print(f"i = {i}, tau**i = {tau_power}, tau_power_G1 = {tau_power_G1}")
    return powers_of_tau


# create [τ^0 G1],[τ^1 G1][τ^2 G1]...[τ^degree +1 G1]
powers_of_tau_testing = compute_powers_of_tau_test(tau_testing, polynomial.degree)

# reverse the polynomial coefficients for 4x^4 + 10x^3 + 7x^2 + 7x + 2  to [2, 7, 7, 10, 4]
reversed_polynomial_coeffs = polynomial.coeffs[::-1]
print(f"Reversed polynomial coefficients: {reversed_polynomial_coeffs}")

def inner_product_test(powers_of_tau, coeffs):
    """Computes the elliptic curve point representing the inner product of the reversed polynomial coefficients
    and the powers of tau points."""
    accumulator = Z1
    for tau_power, coeff in zip(powers_of_tau, coeffs):
        term = multiply(tau_power, int(coeff))
        accumulator = add(accumulator, term)
    return accumulator

assert inner_product_test(powers_of_tau_testing, reversed_polynomial_coeffs) == p_eval_at_tau_G1, "doesn't match"

Polynomial: 4x^4 + 10x^3 + 7x^2 + 7x + 2
Polynomial evaluated at tau: 722942
i = 0, tau**i = 1, tau_power_G1 = (1, 2)
i = 1, tau**i = 20, tau_power_G1 = (18947110137775984544896515092961257947872750783784269176923414004072777296602, 12292085037693291586083644966434670280746730626861846747147579999202931064992)
i = 2, tau**i = 400, tau_power_G1 = (16262199471205794413544947826745938654132104752637586692048329713311590397011, 13296900385261935021718889695689394625708483652039722230815936262285054528714)
i = 3, tau**i = 8000, tau_power_G1 = (21603600070689675766438470661345954782419355034652174505468210225883925863279, 15787091953565760722773063158476721787069408761080596737736006929439659337677)
i = 4, tau**i = 160000, tau_power_G1 = (3791913980001525405070663195453841654293855276471519589821575313643995787424, 2219850731288481436925303713906758446890789653022769553096390029843417460412)
Reversed polynomial coefficients: [ 2  7  7 10  4]


In [5]:
x = GF(5)#random.randint(1,10)
y = GF(10)#random.randint(1,15)

v1 = x * x
v2 = v1 * x
v3 = y * y
v4 = v2 * y
out = v2 + GF(2) * v4 - GF(5) * x * v3 - GF(3) * y + GF(2)

L = GF(np.array([
    [0,0,1,0,0,0,0,0],
    [0,0,0,0,1,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,curve_order - 5,0,0,0,0,0] # 113 - 5 = 108
    ]))

R = GF(np.array([
    [0,0,1,0,0,0,0,0],
    [0,0,1,0,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,0,1,0]
    ]))

O = GF(np.array([
    [0,0,0,0,1,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,1],
    [GF(curve_order-2),1,0,3,0,GF(curve_order-1),0,GF(curve_order-2)] # 113 - 2 = 111
    ]))

# in R1CS # m = number of columns # n = number of rows
r1cs_m = L.shape[1]
r1cs_n = L.shape[0]
print(f"m={r1cs_m}, n={r1cs_n}")

witness = GF(np.array([GF(1),out,x,y,v1,v2,v3,v4]))
result = O.dot(witness) == np.multiply(L.dot(witness),R.dot(witness))
assert result.all(), "result contains an inequality"
print(f"verified")

m=8, n=5
verified


In [6]:
# R1CS => QAP
# in QAP m = number of polynomials. n = degree of target polynomial
tau = GF(20)
target_coeffs = GF(np.array([1,2,3,4,5]))
tx_poly = galois.Poly(np.flip(np.polynomial.polynomial.polyfromroots([1,2,3,4,5])).astype(int),  field=GF)

def matrix_to_vector_of_polynomials(matrix):
    return np.apply_along_axis(interpolate_columns, 0, matrix)

def interpolate_columns(column):
    return galois.lagrange_poly(target_coeffs, column)

# Once interpolation is finished, L, R, O = U.a, V.a, W.a
U_vector_of_polys = matrix_to_vector_of_polynomials(L)
V_vector_of_polys = matrix_to_vector_of_polynomials(R)
W_vector_of_polys = matrix_to_vector_of_polynomials(O)

qap_m = len(U_vector_of_polys)
qap_n = tx_poly.degree # not sure if this is right

# these 2 functions demonstrate that we can either
# 1) multiply polynomials and witness together to 1 polynomial, then evaluate at tau
def aggregate_vector_of_polynomials_with_witness(vector_of_poly, witness, verbose=False):
    sum = GF(0)
    for i in range(len(vector_of_poly)):
        if verbose:
            print(f"scaling witness {witness[i]} by {vector_of_poly[i]}")
        sum += vector_of_poly[i]*witness[i]
    return sum

# 2) evaluate each polynomial at tau, then multiply by witness
def aggregate_vector_of_polynomials_with_witness_at_tau(vector_of_poly, witness, verbose=False):
    sum = GF(0)
    for i in range(len(vector_of_poly)):
        if verbose:
            print(f"scaling witness {witness[i]} by {vector_of_poly[i]}")
        sum += vector_of_poly[i](tau) * witness[i]
    return sum

# This is a polynomial of the inner product of Matrix and Witness
Ua_poly = aggregate_vector_of_polynomials_with_witness(U_vector_of_polys, witness, verbose=True)
Va_poly = aggregate_vector_of_polynomials_with_witness(V_vector_of_polys, witness)
Wa_poly = aggregate_vector_of_polynomials_with_witness(W_vector_of_polys, witness)

# We can now create h(x)t(x)
# we test if the polynomial is the same
hx_poly, remainder = divmod(((Ua_poly*Va_poly) - Wa_poly), tx_poly)
assert remainder == 0, "remainder is not 0"
Ua_poly * Va_poly == Wa_poly + hx_poly * tx_poly
print(f"lhs = rhs so they are the same")

# then test if the polynomial evaluated at tau is the same (schwartz zippel)
tau = GF(9)
Ua_poly(tau) * Va_poly(tau) == Wa_poly(tau) + hx_poly(tau) * tx_poly(tau)
print(f"when evaluated at random number tau, they are the same")

scaling witness 1 by 0
scaling witness 97 by 0
scaling witness 5 by 3648040478639879203707734290876212514758060733402672390616367364429301415936x^4 + 10944121435919637611123202872628637544274182200208017171849102093287904247810x^3 + 7296080957279758407415468581752425029516121466805344781232734728858602831868x^2 + 4x
scaling witness 10 by 16416182153879456416684804308942956316411273300312025757773653139931856371713x^4 + 21888242871839275222246405745257275088548364400416034343698204186575808495614x^3 + 16416182153879456416684804308942956316411273300312025757773653139931856371725x^2 + 10944121435919637611123202872628637544274182200208017171849102093287904247789x + 10
scaling witness 25 by 3648040478639879203707734290876212514758060733402672390616367364429301415936x^4 + 18240202393199396018538671454381062573790303667013361953081836822146507079683x^3 + 18240202393199396018538671454381062573790303667013361953081836822146507079671x^2 + 364804047863987920370773429087621251475806073340267239061

In [7]:
# We use trusted setup to hide polynomial coeffs of U, V, W, h, t
from functools import reduce
tau = GF(20)


# Trusted Setup
# Generate 1) Powers of tau G1, G2. 2) Powers of tau for target polynomial G1. 3) target polynomial evaluated at tau
powers_of_tau_G1_points = [multiply(G1, int(tau) ** i) for i in range(0,qap_n)]
powers_of_tau_G2_points = [multiply(G2, int(tau) ** i) for i in range(0,qap_n)]
tx_powers_of_tau_G1_points = [multiply(G1, int(tau) ** i) for i in range(0,tx_poly.degree)]
t_of_tau_scalar = tx_poly(tau)
tx_powers_of_tau_G1_eval_at_tau = [multiply(tx_powers_of_tau_G1_points[i], int(t_of_tau_scalar)) for i in range(0, len(tx_powers_of_tau_G1_points))]

# Quick sanity test - this is not encrypted and wouldn't happen in real life
Ua_tau_scalar = Ua_poly(tau)
Va_tau_scalar = Va_poly(tau)
Wa_tau_scalar = Wa_poly(tau)
hx_tau_scalar = hx_poly(tau)
tx_tau_scalar = tx_poly(tau)
print(f"Ua(tau) = {Ua_tau_scalar}")
print(f"Va(tau) = {Va_tau_scalar}")
print(f"Wa(tau) = {Wa_tau_scalar}")
print(f"hx(tau) = {hx_tau_scalar}")
print(f"tx(tau) = {tx_tau_scalar}")
print(f"ht= {hx_tau_scalar * tx_tau_scalar}")

assert Ua_tau_scalar * Va_tau_scalar == Wa_tau_scalar + hx_tau_scalar * tx_tau_scalar, "Ua_tau * Va_tau != Wa_tau + hx_tau * tx_tau"

def inner_product(powers_of_tau, coeffs):
    return reduce(add, (multiply(point, int(coeff)) for point, coeff in zip(powers_of_tau, coeffs)), Z1)

def inner_product_iterable(powers_of_tau, coeffs):
    # print(len(powers_of_tau_G1), " eq? ", len(coeffs))
    result = Z1
    for i, (point, coeff) in enumerate(zip(powers_of_tau, coeffs)):
        # print(f"Iteration: {i}, Coeff: {coeff}, Point: {point}")
        result = add(result, multiply(point, int(coeff)))
    return result
 
Ua_tau_G1_point = inner_product_iterable(powers_of_tau_G1_points, Ua_poly.coeffs[::-1])
Va_tau_G2_point = inner_product_iterable(powers_of_tau_G2_points, Va_poly.coeffs[::-1])
Wa_tau_G1_point = inner_product_iterable(powers_of_tau_G1_points, Wa_poly.coeffs[::-1])

# we use from [o:hx.degree + 1] because we need to cover all hx coeffs, not just exponentiated
ht_tau_G1_point = inner_product_iterable(tx_powers_of_tau_G1_eval_at_tau[0:hx_poly.degree + 1], hx_poly.coeffs[::-1])

AB = pairing(Va_tau_G2_point, neg(Ua_tau_G1_point))
CD = pairing(G2, add(Wa_tau_G1_point, ht_tau_G1_point))
final_exponentiate(AB * CD) == FQ12.one()

Ua(tau) = 21888242871839275222246405745257275088548364400416034343698204186575806479342
Va(tau) = 398150
Wa(tau) = 21888242871839275222246405745257275088548364400416034343698204186575781150367
hx(tau) = 8512094450159718141984713345377829201102141711272902244771523850335036061883
tx(tau) = 1395360
ht= 21888242871839275222246405745257275088548364400416034343698204185773055949617


True

In [14]:
# shift lhs and rhs by alpha and beta
# Trusted Setup
alpha_scalar = GF(2)
beta_scalar = GF(3)
AlphaG1_point = multiply(G1, int(alpha_scalar))
BetaG2_point = multiply(G2, int(beta_scalar))

# # reminder
# # qap_m = len(U_vector_of_poly_x). qap_n = target_polynomial.degree # not sure if this is right

# encrypted evaluation: shifts the U and V polynomial by alpha and beta, evaluates at powers of tau
# Trusted Setup completes this with unencrypted tau, alpha, beta values
# Powers of tau for C 
def compute_powers_of_tau_C_G1_points(U_vector_of_polys, V_vector_of_polys, W_vector_of_polys, tau, alpha_scalar, beta_scalar):
    powers_of_tau_C_G1_points = []
    for i in range(len(U_vector_of_polys)):
        iterator_u_scalar = U_vector_of_polys[i](tau)
        iterator_v_scalar = V_vector_of_polys[i](tau)
        iterator_w_scalar = W_vector_of_polys[i](tau)
        sum_scalar = int(beta_scalar * iterator_u_scalar + alpha_scalar * iterator_v_scalar + iterator_w_scalar)
        print(f"sum polynomial for iterator {i} = {sum_scalar}")
        powers_of_tau_C_G1_points.append(multiply(G1, sum_scalar))
    return powers_of_tau_C_G1_points

powers_of_tau_C_G1_points = compute_powers_of_tau_C_G1_points(U_vector_of_polys, V_vector_of_polys, W_vector_of_polys, tau, alpha_scalar, beta_scalar)
powers_of_tau_C_G1_points_scaled_with_witness = [multiply(powers_of_tau_C_G1_points[i], int(witness[i])) for i in range(len(powers_of_tau_C_G1_points))]
print(f"powers of tau C G1 points scaled with witness= {powers_of_tau_C_G1_points_scaled_with_witness}")


def inner_product_iterable_test(powers_of_tau, coeffs):
    result = Z1
    for i, (point, coeff) in enumerate(zip(powers_of_tau, coeffs)):
        result = add(result, multiply(point, int(coeff)))
    return result

def powers_of_tau_innerproduct_poly_i(powers_of_tau, vector_of_polys, witness):
    accumulator = Z1
    for i in range(0, len(vector_of_polys)):
        iterator_point = inner_product_iterable_test(powers_of_tau, vector_of_polys[i].coeffs[::-1])
        accumulator = add(accumulator, multiply(iterator_point, int(witness[i])))
    return accumulator    

A_G1_point = powers_of_tau_innerproduct_poly_i(powers_of_tau_G1_points, U_vector_of_polys, witness)
B_G2_point = powers_of_tau_innerproduct_poly_i(powers_of_tau_G2_points, V_vector_of_polys, witness)

A_G1_point = add(A_G1_point, AlphaG1_point)
B_G2_point = add(B_G2_point, BetaG2_point)

# C_G1_point = Z1
# for i in range(len(powers_of_tau_C_G1_points)):
#     C_G1_point = add(C_G1_point, powers_of_tau_C_G1_points[i])

def sum_points_up_to_length(points):
    result_point = Z1
    for i in range(len(points)):
        result_point = add(result_point, points[i])
    return result_point

C_G1_point = sum_points_up_to_length(powers_of_tau_C_G1_points_scaled_with_witness)
C_G1_point_httau = add(C_G1_point, ht_tau_G1_point)

print("A_G1_point", A_G1_point)
print("B_G2_point", B_G2_point)
print("C_G1_point", C_G1_point_httau)
print("alphaG1_point", AlphaG1_point)
print("betaG2_point", BetaG2_point)

AB = pairing(B_G2_point, neg(A_G1_point))
CD = pairing(G2, C_G1_point_httau)
EF = pairing(BetaG2_point, AlphaG1_point)
final_exponentiate(AB * CD * EF) == FQ12.one()

sum polynomial for iterator 0 = 21888242871839275222246405745257275088548364400416034343698204186575808487865
sum polynomial for iterator 1 = 3876
sum polynomial for iterator 2 = 21888242871839275222246405745257275088548364400416034343698204186575808426937
sum polynomial for iterator 3 = 85158
sum polynomial for iterator 4 = 21888242871839275222246405745257275088548364400416034343698204186575808459917
sum polynomial for iterator 5 = 21888242871839275222246405745257275088548364400416034343698204186575808435216
sum polynomial for iterator 6 = 28272
sum polynomial for iterator 7 = 21888242871839275222246405745257275088548364400416034343698204186575808473330
powers of tau C G1 points scaled with witness= [(18412086319910179752494782965811456442455794258224578119780579512176567504706, 10558562959246852549340863372112227129585961702953419993535490858069696891127), (3241640664343904866761662067828845588248524718703942440965736023541316835185, 50021613405133204413172266512635456730613975697931

True

In [17]:
# separate public and private inputs
# public input, computed by verifier
public_witness_length = 2
public_input = []

# Trusted Setup
gamma = GF(4)
delta = GF(5)
gamma_inv = pow(int(gamma),-1, curve_order)
delta_inv = pow(int(delta),-1, curve_order)
GammaG2 = multiply(G2, int(gamma))
DeltaG2 = multiply(G2, int(delta))
DeltaG1 = multiply(G1, int(delta))

# gamma is public, trusted setup can compute with public witness
def compute_powers_of_tau_C_G1_points_gamma_delta(
        U_vector_of_polys, 
        V_vector_of_polys, 
        W_vector_of_polys, 
        tau, 
        alpha_scalar, 
        beta_scalar,
        start_index,
        end_index
        ):
    powers_of_tau_C_points = []
    for i in range(start_index,end_index):
        iterator_u_scalar = U_vector_of_polys[i](tau)
        iterator_v_scalar = V_vector_of_polys[i](tau)
        iterator_w_scalar = W_vector_of_polys[i](tau)
        sum_scalar = int(beta_scalar * iterator_u_scalar + alpha_scalar * iterator_v_scalar + iterator_w_scalar)
        if(start_index == 0):
            sum_scalar = sum_scalar * int(gamma_inv)
        else:
            sum_scalar = sum_scalar * int(delta_inv)
        powers_of_tau_C_points.append(multiply(G1, sum_scalar))
    return powers_of_tau_C_points


powers_of_tau_C_G1_points_gamma = compute_powers_of_tau_C_G1_points_gamma_delta(
    U_vector_of_polys, 
    V_vector_of_polys, 
    W_vector_of_polys, 
    tau, 
    alpha_scalar, 
    beta_scalar,
    0,
    public_witness_length
    )

powers_of_tau_C_G1_points_delta = compute_powers_of_tau_C_G1_points_gamma_delta(
    U_vector_of_polys, 
    V_vector_of_polys, 
    W_vector_of_polys, 
    tau, 
    alpha_scalar, 
    beta_scalar,
    public_witness_length,
    len(U_vector_of_polys)
)

print("powers_of_tau_C_G1_points_gamma", powers_of_tau_C_G1_points_gamma)
print("powers_of_tau_C_G1_points_delta", powers_of_tau_C_G1_points_delta)
assert(len(powers_of_tau_C_G1_points_gamma) + len(powers_of_tau_C_G1_points_delta) == len(U_vector_of_polys))

# scale with witness values
# powers_of_tau_C_G1_point_scaled_with_witness_gamma = [multiply(powers_of_tau_C_G1_points_gamma[i], int(witness[i])) for i in range(len(powers_of_tau_C_G1_points_gamma))]

def inner_product_iterable(powers_of_tau, coeffs):
    # print(len(powers_of_tau_G1), " eq? ", len(coeffs))
    result = Z1
    for i, (point, coeff) in enumerate(zip(powers_of_tau, coeffs)):
        # print(f"Iteration: {i}, Coeff: {coeff}, Point: {point}")
        result = add(result, multiply(point, int(coeff)))
    return result

powers_of_tau_C_G1_point_scaled_with_witness_delta = inner_product_iterable(
    powers_of_tau_C_G1_points_delta, 
    witness[public_witness_length:len(witness)]
    )

powers_of_tau_C_G1_point_scaled_with_witness_gamma = inner_product_iterable(
    powers_of_tau_C_G1_points_gamma, 
    witness[0:public_witness_length]
    )


ht_tau_G1_point_delta = multiply(ht_tau_G1_point, int(delta_inv))
C_G1_point_delta = add(powers_of_tau_C_G1_point_scaled_with_witness_delta, ht_tau_G1_point_delta)

print("powers_of_tau_C_G1_point_scaled_with_witness_gamma", powers_of_tau_C_G1_point_scaled_with_witness_gamma)
print("powers_of_tau_C_G1_point_scaled_with_witness_delta", powers_of_tau_C_G1_point_scaled_with_witness_delta)
print("C_G1_point_delta", C_G1_point_delta)

print("Gamma G2", GammaG2)
print("powers_of_tau_C_G1_point_scaled_with_witness_gamma",powers_of_tau_C_G1_point_scaled_with_witness_gamma)
print("Delta G2", DeltaG2)
print("C G1", C_G1_point_delta)

AB = pairing(B_G2_point, neg(A_G1_point))
CD = pairing(BetaG2_point, AlphaG1_point)
EF = pairing(GammaG2, powers_of_tau_C_G1_point_scaled_with_witness_gamma)
GH = pairing(DeltaG2,C_G1_point_delta)
final_exponentiate(AB * CD * EF * GH) == FQ12.one()

powers_of_tau_C_G1_points_gamma [(15470831961688208669697575643418221048181962981001546150468987941769939214029, 18777991423107314776482652904700650180890186299459272499931369322486171899854), (1540147615742092855122362390273215696202300989980535937807544369063691019287, 916302540883947080468652904864832717784863855769927076757632797189693955321)]
powers_of_tau_C_G1_points_delta [(16524110854952697577386273560426967717495967530452967658401010590431306966138, 16823085444673229695753433392858734602211304308546827796553415028031795593245), (12574349950908472009468905730079111416172122556667425914633421861623499981593, 17739771797728650120276270997904923612762351305448443717909117692436697294124), (6731630599068668007840269770938900222530134894973364651665853959181471539119, 10225484116681274662354198423147565710024010125244647183213660508170839009807), (15978187043361717096031923363018135656566106143427870928587437386104352678546, 3207896598691833441170806557841821196905561228195395022789

True

In [10]:
# introducing proving phase shift to protect witness with r, s
r = GF(6)
s = GF(8)
r_delta_G1 = multiply(DeltaG1, int(r))
s_delta_G2 = multiply(DeltaG2, int(s))
s_delta_G1 = multiply(DeltaG1, int(s))

B_G1_point = powers_of_tau_innerproduct_poly_i(powers_of_tau_G1_points, V_vector_of_polys, witness)
BetaG1_point = multiply(G1, int(beta_scalar))
B_G1_point = add(B_G1_point, BetaG1_point)

A_G1_point_rd = add(A_G1_point, r_delta_G1)
B_G2_point_sd = add(B_G2_point, s_delta_G2)
B_G1_point_sd = add(B_G1_point, s_delta_G1)

# C_G1_point_httau
A_G1_point_s = multiply(A_G1_point_rd, int(s))
B_G1_point_r = multiply(B_G1_point_sd, int(r))
Delta_G1_point_neg_rs = multiply(DeltaG1, int(-r*s))
C_G1_point_rsd = add(add(C_G1_point_httau,A_G1_point_s),add(B_G1_point_r, Delta_G1_point_neg_rs))

AB = pairing(B_G2_point_sd, neg(A_G1_point_rd))
CD = pairing(G2, C_G1_point_rsd)
final_exponentiate(AB * CD) == FQ12.one()

False